### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import urllib.parse
from pandas import json_normalize


In [2]:
# Set environment variables from the .env in the local environment
load_dotenv('example.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
encoded_filter_query = urllib.parse.quote(filter_query)



# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

nyt_query_url = (
    
    f"{url}api-key={nyt_api_key}&sort={sort}&fl={field_list}&fq={encoded_filter_query}&begin_date={begin_date}&end_date={end_date}"
)

nyt_query_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=DmmtDTHtncGjdBorA7zpNJ5TRyAkN9Go&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&fq=section_name%3A%22Movies%22%20AND%20type_of_material%3A%22Review%22%20AND%20headline%3A%22love%22&begin_date=20130101&end_date=20230531'

In [4]:
response = requests.get(nyt_query_url)
response

<Response [200]>

In [5]:
# Create an empty list to store the reviews

nyt_movie_reviews = []

# loop through pages 0-19
# create query with a page number
# Make a "GET" request and retrieve the JSON
# Add a twelve second interval between queries to stay within API query limits
# loop through the reviews["response"]["docs"] and append each review to the list
# Print the page that was just retrieved
# Print the page number that had no results then break from the loop
# Try and save the reviews to the reviews_list

for page in range(20):
    
    page_url = f"{nyt_query_url}&page = {page}"
    response = requests.get(page_url)
    time.sleep(12)
    
    if response.status_code == 200:
        data = response.json()
        articles = data.get('response', {}).get('docs', [])
        nyt_movie_reviews.append(articles)
        print(f"Retrieved page {page}")
    
    else:
        print(f"Failed to fetch data from page {page}")
        break
        
nyt_movie_reviews = [review for page_reviews in nyt_movie_reviews for review in page_reviews]

# print(f"Total number of reviews fetched: {len(nyt_movie_reviews)}")


Retrieved page 0
Retrieved page 1
Retrieved page 2
Retrieved page 3
Retrieved page 4
Retrieved page 5
Retrieved page 6
Retrieved page 7
Retrieved page 8
Retrieved page 9
Retrieved page 10
Retrieved page 11
Retrieved page 12
Retrieved page 13
Retrieved page 14
Retrieved page 15
Retrieved page 16
Retrieved page 17
Retrieved page 18
Retrieved page 19


In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

nyt_movie_reviews_preview_results = nyt_movie_reviews[:5]

nyt_movie_reviews_preview_results_formatted = json.dumps(nyt_movie_reviews_preview_results, indent = 4)

print(nyt_movie_reviews_preview_results_formatted)

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()

nyt_movie_reviews_df = json_normalize(nyt_movie_reviews)

nyt_movie_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
196,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
197,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
198,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None


In [8]:
# Extract the title from the "headline.main" column and save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# Instructor reviewed during class and approved using a different logic than advised: 
#lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]

def extract_title(headline):
    title_start_index = headline.find('\u2018') + 1
    title_end_index = headline.find('\u2019 Review') + len(' Review')
    title = headline[title_start_index: title_end_index]
    return title.strip()

nyt_movie_reviews_df['title'] = nyt_movie_reviews_df['headline.main'].apply(extract_title)

nyt_movie_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries’ Revie
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever’ Revie
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love’ Revie
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children’ Revie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves’ Revie
196,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story’ Revie
197,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One’ Revie
198,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine’ Revie


In [11]:
#Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string

nyt_movie_reviews_df['keywords']

# nyt_movie_reviews_df['keywords'] = nyt_movie_reviews_df['keywords'].apply(extract_keywords)

# nyt_movie_reviews_df

#TODO: Fix bug that makes certain keywords and title of movies not appear.


0      [{'name': 'subject', 'value': 'Movies', 'rank'...
1      [{'name': 'subject', 'value': 'Movies', 'rank'...
2      [{'name': 'subject', 'value': 'Movies', 'rank'...
3      [{'name': 'subject', 'value': 'Movies', 'rank'...
4      [{'name': 'subject', 'value': 'Movies', 'rank'...
                             ...                        
195    [{'name': 'subject', 'value': 'Movies', 'rank'...
196    [{'name': 'subject', 'value': 'Documentary Fil...
197    [{'name': 'subject', 'value': 'Movies', 'rank'...
198    [{'name': 'subject', 'value': 'Movies', 'rank'...
199    [{'name': 'subject', 'value': 'Documentary Fil...
Name: keywords, Length: 200, dtype: object

In [12]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

nyt_movie_reviews_df_title_list = nyt_movie_reviews_df['title'].to_list()

nyt_movie_reviews_df_title_list

['The Attachment Diaries’ Revie',
 '',
 'You Can Live Forever’ Revie',
 'A Tourist’s Guide to Love’ Revie',
 'Other People’s Children’ Revie',
 'One True Loves’ Revie',
 'The Lost Weekend: A Love Story’ Revie',
 'A Thousand and One’ Revie',
 'Your Place or Mine’ Revie',
 'Love in the Time of Fentanyl’ Revie',
 'The Attachment Diaries’ Revie',
 '',
 'You Can Live Forever’ Revie',
 'A Tourist’s Guide to Love’ Revie',
 'Other People’s Children’ Revie',
 'One True Loves’ Revie',
 'The Lost Weekend: A Love Story’ Revie',
 'A Thousand and One’ Revie',
 'Your Place or Mine’ Revie',
 'Love in the Time of Fentanyl’ Revie',
 'The Attachment Diaries’ Revie',
 '',
 'You Can Live Forever’ Revie',
 'A Tourist’s Guide to Love’ Revie',
 'Other People’s Children’ Revie',
 'One True Loves’ Revie',
 'The Lost Weekend: A Love Story’ Revie',
 'A Thousand and One’ Revie',
 'Your Place or Mine’ Revie',
 'Love in the Time of Fentanyl’ Revie',
 'The Attachment Diaries’ Revie',
 '',
 'You Can Live Forever’ Revi

### Access The Movie Database API

In [13]:
# Prepare The Movie Database query

# The search query is used to find the movie ID from the search by title. Most of this query is included in your starter code, as follows, but you will need to include the movie title in the query.
# Prepare The Movie Database query
# url = "https://api.themoviedb.org/3/search/movie?query="
# tmdb_key_string = "&api_key=" + tmdb_api_key

url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = "love"
encoded_filter_query = urllib.parse.quote(filter_query)


tmdb_query_url = f"{url}{encoded_filter_query}{tmdb_key_string}"
tmdb_query_url




'https://api.themoviedb.org/3/search/movie?query=love&api_key=efe1c99f6c82f45a286b6cbf5d9e64ab'

In [14]:
response = requests.get(tmdb_query_url)
response

<Response [200]>

In [16]:
# Loop through the titles
# Check if we need to sleep before making a request
# Add 1 to the request counter
# Include a try clause to search for the full movie details.
# Perform a "GET" request for The Movie Database
# Get movie id
# Make a request for a the full movie details
# Execute "GET" request with url
# Extract the genre names into a list
# Extract the spoken_languages' English name into a list
# Extract the production_countries' name into a list
# Add the relevant data to a dictionary and
# append it to the tmdb_movies_list list
# Print out the title that was found
# Use the except clause to print out a statement if a movie is not found.



# Create an empty list to store the results
tmdb_movie_reviews = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0

# Loop through the titles
for title in nyt_movie_reviews_df_title_list:
    # Skip empty titles
    if not title.strip():
        print("Skipping empty title")
        continue

    # Process the title to handle special characters and formatting
    formatted_title = title.strip().rstrip("’ Revie")  

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(10)  # Sleep for 10 seconds every 50 requests

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    response = requests.get(f"{url}{urllib.parse.quote(formatted_title)}{tmdb_key_string}")

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Check if results are present
        if "results" in response.json() and response.json()["results"]:
            # Get movie id
            movie_id = response.json()["results"][0]["id"]

            # Make a request for the full movie details
            movie_details_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}")

            # Execute "GET" request with url
            movie_details = movie_details_response.json()

            # Extract the genre names into a list
            genres = [genre["name"] for genre in movie_details["genres"]]

            # Extract the spoken_languages' English name into a list
            spoken_languages = [lang["english_name"] for lang in movie_details["spoken_languages"]]

            # Extract the production_countries' name into a list
            production_countries = [country["name"] for country in movie_details["production_countries"]]

            # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
            movie_info = {
                "title": title,
                "original_title": movie_details["original_title"],
                "budget": movie_details["budget"],
                "original_language": movie_details["original_language"],
                "homepage": movie_details["homepage"],
                "overview": movie_details["overview"],
                "popularity": movie_details["popularity"],
                "runtime": movie_details["runtime"],
                "revenue": movie_details["revenue"],
                "release_date": movie_details["release_date"],
                "vote_average": movie_details["vote_average"],
                "vote_count": movie_details["vote_count"],
                "genres": genres,
                "spoken_languages": spoken_languages,
                "production_countries": production_countries
            }
            
            tmdb_movie_reviews.append(movie_info)

            # Print out the title that was found
            print(f"Found movie: {title}")
        else:
            print(f"Movie not found: {title}")

    except Exception as e:
        print(f"An error occurred while processing {title}: {e}")


Found movie: The Attachment Diaries’ Revie
Skipping empty title
Found movie: You Can Live Forever’ Revie
Found movie: A Tourist’s Guide to Love’ Revie
Found movie: Other People’s Children’ Revie
Found movie: One True Loves’ Revie
Found movie: The Lost Weekend: A Love Story’ Revie
Found movie: A Thousand and One’ Revie
Found movie: Your Place or Mine’ Revie
Found movie: Love in the Time of Fentanyl’ Revie
Found movie: The Attachment Diaries’ Revie
Skipping empty title
Found movie: You Can Live Forever’ Revie
Found movie: A Tourist’s Guide to Love’ Revie
Found movie: Other People’s Children’ Revie
Found movie: One True Loves’ Revie
Found movie: The Lost Weekend: A Love Story’ Revie
Found movie: A Thousand and One’ Revie
Found movie: Your Place or Mine’ Revie
Found movie: Love in the Time of Fentanyl’ Revie
Found movie: The Attachment Diaries’ Revie
Skipping empty title
An error occurred while processing You Can Live Forever’ Revie: HTTPSConnectionPool(host='api.themoviedb.org', port=443)

ReadTimeout: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Read timed out. (read timeout=None)

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

for movie in tmdb_movie_reviews:
    print(json.dumps(movie, indent = 4))





# tmdb_movie_reviews_preview_results = tmdb_movie_reviews[:5]

# tmdb_movie_reviews_preview_results_formatted = json.dumps(tmdb_movie_reviews_preview_results, indent = 4)

# print(tmdb_movie_reviews_preview_results_formatted)


In [ ]:
# Convert the results to a DataFrame
tmdb_movie_reviews_df = json_normalize(tmdb_movie_reviews)

tmdb_movie_reviews_df

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title

merged_df = pd.merge(nyt_movie_reviews_df, tmdb_movie_reviews_df, on="title", how="inner")

merged_df

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
